In [8]:
import os
import logging
import time
import random
from anthropic import Anthropic

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Initialize the Anthropic client with the hardcoded API key
client = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))

# Directories for input notes and output files
input_dir = '/Users/vineetreddy/Documents/Github/EngelOutcomeNLP/data/Clinical Notes/'
output_dir = '/Users/vineetreddy/Documents/Github/EngelOutcomeNLP/output'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# List all clinical note files in the input directory
note_files = os.listdir(input_dir)

# Define the Engel Outcome Scale criteria
criteria = '''Engel Outcome Scale:
Class I: Free of disabling seizures

IA: Completely seizure-free since surgery
IB: Non disabling simple partial seizures only since surgery
IC: Some disabling seizures after surgery, but free of disabling seizures for at least 2 years
ID: Generalized convulsions with antiepileptic drug withdrawal only

Class II: Rare disabling seizures (“almost seizure-free”)

IIA: Initially free of disabling seizures but has rare seizures now
IIB: Rare disabling seizures since surgery
IIC: More than rare disabling seizures after surgery, but rare seizures for at least 2 years
IID: Nocturnal seizures only

Class III: Worthwhile improvement

IIIA: Worthwhile seizure reduction
IIIB: Prolonged seizure-free intervals amounting to greater than half the follow-up period, but not less than 2 years

Class IV: No worthwhile improvement

IVA: Significant seizure reduction
IVB: No appreciable change
IVC: Seizures worse.'''

for note_file in note_files:
    try:
        # Construct full file paths
        input_path = os.path.join(input_dir, note_file)
        output_path = os.path.join(output_dir, f'output_{note_file}')

        # Read the clinical note content
        with open(input_path, 'r', encoding='utf-8') as f:
            clinical_note = f.read()

        # Create the message content
        message_content = f"""Please read the following clinical note and, using the Engel Outcome Scale criteria provided below, assign an Engel score (number and letter) to the patient. Make sure to output the Engel Score, even if you are unsure. Then, in a separate section, provide a detailed explanation for your choice, clearly explaining your clinical reasoning. Ignore the fact that the patient is not post-surgery.

{criteria}

Clinical Note:
{clinical_note}"""

        # Format the prompt as per Anthropic API requirements
        prompt = f"\n\nHuman: {message_content}\n\nAssistant:"

        # Call the Anthropic API using the completions endpoint
        response = client.completions.create(
            model="claude-2",
            max_tokens_to_sample=2000,
            temperature=0.7,
            prompt=prompt
        )

        # Extract the completion text
        output_text = response.completion.strip()

        # Save the output to a file
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(output_text)

        logging.info(f"Processed '{note_file}' and saved the output to '{output_path}'.")

        # Optional: Pause between requests to respect rate limits
        time.sleep(random.uniform(2, 5))  # Sleep between 2 to 5 seconds

    except KeyboardInterrupt:
        logging.info("Process interrupted by user.")
        break
    except Exception as e:
        logging.error(f"Failed to process '{note_file}': {e}")
        if '429' in str(e):
            logging.info("Rate limit exceeded. Sleeping for 60 seconds.")
            time.sleep(60)
            continue


INFO: HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
INFO: Processed 'EA0135.txt' and saved the output to '/Users/vineetreddy/Documents/Github/EngelOutcomeNLP/output/output_EA0135.txt'.
INFO: HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
INFO: Processed 'EA0121.txt' and saved the output to '/Users/vineetreddy/Documents/Github/EngelOutcomeNLP/output/output_EA0121.txt'.
INFO: HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
INFO: Processed 'EA0109.txt' and saved the output to '/Users/vineetreddy/Documents/Github/EngelOutcomeNLP/output/output_EA0109.txt'.
INFO: HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
INFO: Processed 'EA0041.txt' and saved the output to '/Users/vineetreddy/Documents/Github/EngelOutcomeNLP/output/output_EA0041.txt'.
INFO: HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
INFO: Processed 'EA0055.txt' and saved the output to '/Users/vi